In [1]:
import os
import random

import openai
import tiktoken

from pathlib import Path
from pprint import pprint
from getpass import getpass

from rich.markdown import Markdown
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,  # for exponential backoff
)
import wandb
from wandb.integration.openai import autolog

In [2]:
if os.getenv("OPENAI_API_KEY") is None:
    if any(["VSCODE" in x for x in os.environ.keys()]):
        print(
            "Please enter password in the VS Code prompt at the top of your VS Code window!"
        )
    os.environ["OPENAI_API_KEY"] = getpass(
        "Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n"
    )
    openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith(
    "sk-"
), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

Please enter password in the VS Code prompt at the top of your VS Code window!
OpenAI API key configured


In [3]:
autolog({"project": "studybot", "job_type": "qapair"})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: hemanthsai7. Use `wandb login --relogin` to force relogin


In [4]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(3))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [5]:
MODEL_NAME = "gpt-3.5-turbo"
# MODEL_NAME = "gpt-4"

In [6]:
system_prompt = "You are a helpful assisstant"
user_prompt = "Generate a support question from a class 10th student about the CBSE class 10th history chapter 'The Rise of Nationalism in Europe'"


def generate_and_print(system_prompt, user_prompt, n=3):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    responses = completion_with_backoff(
        model=MODEL_NAME,
        messages=messages,
        n=n,
    )

    for response in responses.choices:
        generation = response.message.content
        display(Markdown(generation))


generate_and_print(system_prompt, user_prompt)

"Can you provide a brief overview of the major factors that contributed to the rise of nationalism in Europe during
the 19th century, as discussed in the CBSE class 10th history chapter 'The Rise of Nationalism in Europe'?"

"Can you please explain the significance of major nationalist movements in Europe during the 19th century, as      
discussed in the CBSE class 10th history chapter 'The Rise of Nationalism in Europe'?"

"What were the key factors that contributed to the rise of nationalism in Europe during the 19th century, as       
explained in the CBSE class 10th history chapter 'The Rise of Nationalism in Europe'?"